In [1]:
import pandas as pd
import ast

In [2]:
df_trainset = pd.read_csv("data/04_train_set.tsv", sep="\t")
df_trainset

,chr,start,end,strand,class
0,chr1,12227,12612,+,1
1,chr1,12721,13220,+,1
2,chr1,12057,12178,+,1
3,chr1,12697,12974,+,1
4,chr1,13052,13220,+,1
...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0
519030,chrY,25513173,25513588,-,0
519031,chrY,25513745,25516715,-,0
519032,chrY,25525288,25527646,-,0


In [3]:
df_features_dict = {"5": pd.read_csv("data/05_trainset_recount3_feature.tsv", sep="\t"),
                    "6": pd.read_csv("data/06_trainset_repeat_feature.tsv", sep="\t"),
                    "7": pd.read_csv("data/07_trainset_antisense_feature.tsv", sep="\t"),
                    "8": pd.read_csv("data/08_trainset_with_seq.tsv", sep="\t"),
                    "9": pd.read_csv("data/09_trainset_nearest_alt_ss_dist_feature", sep="\t"),
                    "10": pd.read_csv("data/10_trainset_MaxEntScan_feature.tsv", sep="\t"),
                    "11": pd.read_csv("data/11_trainset_splam_feature.tsv", sep="\t"),
                    "12": pd.read_csv("data/12_trainset_conservation_scores_feature.tsv", sep="\t"),
                    "13": pd.read_csv("data/13_recount3_near_ss_with_better_score_feature.tsv", sep="\t"),
                    "14": pd.read_csv("data/14_trainset_BPP_score_feature.tsv", sep="\t")}

In [37]:
df_merged = df_trainset.copy()
df_merged

,chr,start,end,strand,class
0,chr1,12227,12612,+,1
1,chr1,12721,13220,+,1
2,chr1,12057,12178,+,1
3,chr1,12697,12974,+,1
4,chr1,13052,13220,+,1
...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0
519030,chrY,25513173,25513588,-,0
519031,chrY,25513745,25516715,-,0
519032,chrY,25525288,25527646,-,0


In [38]:
# Loop over the dictionary to perform the merging operation
for key in df_features_dict.keys():
    if "class" in str(df_features_dict[key].columns):
        df_features_dict[key] = df_features_dict[key].drop(columns="class", axis=1)
    if len(df_features_dict[key]) == len(df_trainset):
        if key != "10":
                df_merged = pd.concat([df_merged, df_features_dict[key].drop(columns=["chr", "start", "end", "strand"])], axis=1)
        else:
            df_merged = pd.concat([df_merged, df_features_dict[key]], axis=1)
    else:
        df_merged = df_merged.merge(df_features_dict[key], how="left", on=["chr", "start", "end", "strand"])

df_merged

,chr,start,end,strand,class,recount3_score,repeat_features_start_site,repeat_features_end_site,antisense_exon_start_ss,antisense_exon_end_ss,...,recount3_near_start_ss_with_better_score,recount3_near_end_ss_with_better_score,bps,bp_pos,sc_bps,sc_ppt,sc,zsc_bps,zsc_ppt,zsc
0,chr1,12227,12612,+,1,59151,[],[],False,False,...,False,False,GCCCCAG,20.0,0.000232,7.304590e-03,1.694310e-06,4.582576,0.986842,4.582576
1,chr1,12721,13220,+,1,61021,[],[],False,False,...,True,False,GCCTTAG,19.0,0.005862,5.257510e-03,3.082192e-05,4.210821,1.121772,4.429667
2,chr1,12057,12178,+,1,0,[],[],False,False,...,False,False,CCACAAC,19.0,0.003559,7.527838e-07,2.679516e-09,4.050680,1.012740,4.026648
3,chr1,12697,12974,+,1,21,[],[],False,False,...,True,False,GGGTCAT,17.0,0.032313,1.904623e-04,6.154421e-06,3.187796,-1.023767,5.414235
4,chr1,13052,13220,+,1,526,[],[],False,False,...,False,False,GCCTTAG,19.0,0.005862,5.257510e-03,3.082192e-05,4.210821,1.121772,4.429667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0,0,[],[],False,False,...,False,False,GCTTAAA,21.0,0.037471,3.814200e-04,1.429227e-05,4.181171,0.768307,4.144208
519030,chrY,25513173,25513588,-,0,31374,[],[],False,False,...,False,False,AGATAAT,30.0,0.132502,1.618332e-01,2.144329e-02,5.943683,1.104011,5.992466
519031,chrY,25513745,25516715,-,0,67,[],[],False,False,...,True,False,TTCTAAT,24.0,0.217305,1.371833e-05,2.981062e-06,4.472136,1.160504,4.472136
519032,chrY,25525288,25527646,-,0,18958,[],[],False,False,...,False,False,TGGTAAA,22.0,0.048905,3.656019e-03,1.787993e-04,4.123106,1.018607,4.123106


# Populate the repeat_features into one-hots

In [39]:
df_merged['lst_repeat_features_start_site'] = df_merged['repeat_features_start_site'].apply(lambda x: ast.literal_eval(x) if len(x) > 2 else [])
df_merged['lst_repeat_features_end_site'] = df_merged['repeat_features_end_site'].apply(lambda x: ast.literal_eval(x) if len(x) > 2 else [])
df_merged

,chr,start,end,strand,class,recount3_score,repeat_features_start_site,repeat_features_end_site,antisense_exon_start_ss,antisense_exon_end_ss,...,bps,bp_pos,sc_bps,sc_ppt,sc,zsc_bps,zsc_ppt,zsc,lst_repeat_features_start_site,lst_repeat_features_end_site
0,chr1,12227,12612,+,1,59151,[],[],False,False,...,GCCCCAG,20.0,0.000232,7.304590e-03,1.694310e-06,4.582576,0.986842,4.582576,[],[]
1,chr1,12721,13220,+,1,61021,[],[],False,False,...,GCCTTAG,19.0,0.005862,5.257510e-03,3.082192e-05,4.210821,1.121772,4.429667,[],[]
2,chr1,12057,12178,+,1,0,[],[],False,False,...,CCACAAC,19.0,0.003559,7.527838e-07,2.679516e-09,4.050680,1.012740,4.026648,[],[]
3,chr1,12697,12974,+,1,21,[],[],False,False,...,GGGTCAT,17.0,0.032313,1.904623e-04,6.154421e-06,3.187796,-1.023767,5.414235,[],[]
4,chr1,13052,13220,+,1,526,[],[],False,False,...,GCCTTAG,19.0,0.005862,5.257510e-03,3.082192e-05,4.210821,1.121772,4.429667,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0,0,[],[],False,False,...,GCTTAAA,21.0,0.037471,3.814200e-04,1.429227e-05,4.181171,0.768307,4.144208,[],[]
519030,chrY,25513173,25513588,-,0,31374,[],[],False,False,...,AGATAAT,30.0,0.132502,1.618332e-01,2.144329e-02,5.943683,1.104011,5.992466,[],[]
519031,chrY,25513745,25516715,-,0,67,[],[],False,False,...,TTCTAAT,24.0,0.217305,1.371833e-05,2.981062e-06,4.472136,1.160504,4.472136,[],[]
519032,chrY,25525288,25527646,-,0,18958,[],[],False,False,...,TGGTAAA,22.0,0.048905,3.656019e-03,1.787993e-04,4.123106,1.018607,4.123106,[],[]


In [40]:
unique_repeat_features_values_start_site = set()
unique_repeat_features_values_end_site = set()
for idx, row in df_merged.iterrows():
    unique_repeat_features_values_start_site.update(row['lst_repeat_features_start_site'])
    unique_repeat_features_values_end_site.update(row['lst_repeat_features_end_site'])

unique_repeat_features_values_start_site

{'Centromere',
 'Dust',
 'LTRs',
 'Low complexity regions',
 'RNA repeats',
 'Satellite repeats',
 'Simple repeats',
 'Tandem repeats',
 'Type I Transposons/LINE',
 'Type I Transposons/SINE',
 'Type II Transposons',
 'Unknown'}

In [41]:
for repeat_feature in unique_repeat_features_values_start_site:
    df_merged["repeat_features_start_site_"+repeat_feature] = df_merged["repeat_features_start_site"].str.count("'"+repeat_feature+"'").astype(int)


for repeat_feature in unique_repeat_features_values_start_site:
    df_merged["repeat_features_end_site_"+repeat_feature] = df_merged["repeat_features_end_site"].str.count("'"+repeat_feature+"'").astype(int)

df_merged

,chr,start,end,strand,class,recount3_score,repeat_features_start_site,repeat_features_end_site,antisense_exon_start_ss,antisense_exon_end_ss,...,repeat_features_end_site_Type I Transposons/SINE,repeat_features_end_site_RNA repeats,repeat_features_end_site_Low complexity regions,repeat_features_end_site_Simple repeats,repeat_features_end_site_LTRs,repeat_features_end_site_Centromere,repeat_features_end_site_Unknown,repeat_features_end_site_Satellite repeats,repeat_features_end_site_Dust,repeat_features_end_site_Type I Transposons/LINE
0,chr1,12227,12612,+,1,59151,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
1,chr1,12721,13220,+,1,61021,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
2,chr1,12057,12178,+,1,0,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
3,chr1,12697,12974,+,1,21,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
4,chr1,13052,13220,+,1,526,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0,0,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
519030,chrY,25513173,25513588,-,0,31374,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
519031,chrY,25513745,25516715,-,0,67,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0
519032,chrY,25525288,25527646,-,0,18958,[],[],False,False,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df_merged = df_merged.drop(columns=["repeat_features_start_site", "repeat_features_end_site", "lst_repeat_features_start_site", "lst_repeat_features_end_site"])
df_merged

,chr,start,end,strand,class,recount3_score,antisense_exon_start_ss,antisense_exon_end_ss,maxentscan_sequence,start_ss,...,repeat_features_end_site_Type I Transposons/SINE,repeat_features_end_site_RNA repeats,repeat_features_end_site_Low complexity regions,repeat_features_end_site_Simple repeats,repeat_features_end_site_LTRs,repeat_features_end_site_Centromere,repeat_features_end_site_Unknown,repeat_features_end_site_Satellite repeats,repeat_features_end_site_Dust,repeat_features_end_site_Type I Transposons/LINE
0,chr1,12227,12612,+,1,59151,False,False,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,...,0,0,0,0,0,0,0,0,0,0
1,chr1,12721,13220,+,1,61021,False,False,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,...,0,0,0,0,0,0,0,0,0,0
2,chr1,12057,12178,+,1,0,False,False,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,...,0,0,0,0,0,0,0,0,0,0
3,chr1,12697,12974,+,1,21,False,False,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,...,0,0,0,0,0,0,0,0,0,0
4,chr1,13052,13220,+,1,526,False,False,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0,0,False,False,cacGTTGAGGCGCCCAGTGGCGGCCTCACGGGGCAGGGCGAGGGCG...,GT,...,0,0,0,0,0,0,0,0,0,0
519030,chrY,25513173,25513588,-,0,31374,False,False,aggGTAAGATCAGTGCTATTGTCAGAGGAAAAACTCCTGGCCATCA...,GT,...,0,0,0,0,0,0,0,0,0,0
519031,chrY,25513745,25516715,-,0,67,False,False,gctGTAAGTTCCACATTGATTATCATAGGCTAACCATGGGCCAGGC...,GT,...,0,0,0,0,0,0,0,0,0,0
519032,chrY,25525288,25527646,-,0,18958,False,False,tgtGTGTATAAATATCTGGACTTTTTGGTTAAGTAATTATAGTTAA...,GT,...,0,0,0,0,0,0,0,0,0,0


In [43]:
df_merged.to_csv("data/15_trainset_populated.tsv", sep="\t", index=False)